In [1]:
import networkx as nx
import pandas as pd
import numpy as np

import geopandas as gpd
from shapely.geometry import Point, shape, Polygon
import json
import matplotlib.pyplot as plt

import random
from utils import get_osrm_distance
from utils import draw_graph
from utils import calculate_weighted_metrics
from tqdm import tqdm, trange
import os


In [4]:
year = [2011, 2012, 2013, 2014, 2015, 2016, 2017]

for yr in year:

    filename_opt=f"network_graphs_optimized/network_{yr}.graphml"
    # filename_real=f"network_graphs_save/network_{yr}_100.graphml"
    graph_opt = nx.read_graphml(filename_opt)
    m = calculate_weighted_metrics(graph_opt, yr)
    print(m)
    # draw_graph(graph_opt)
    

{'density': 0.12450464304725851, 'average_distance': 53.78601766838495, 'diameter': 9.0, 'average_clustering': 0.704, 'year': 2011, 'total_nodes': 320, 'subnetwork_sizes': [319]}
{'density': 0.14026451623542605, 'average_distance': 50.135825028054924, 'diameter': 9.0, 'average_clustering': 0.766, 'year': 2012, 'total_nodes': 494, 'subnetwork_sizes': [493]}
{'density': 0.14413283546516248, 'average_distance': 48.07905518410057, 'diameter': 9.0, 'average_clustering': 0.828, 'year': 2013, 'total_nodes': 615, 'subnetwork_sizes': [615]}
{'density': 0.12606217756529725, 'average_distance': 53.34244719618865, 'diameter': 10.0, 'average_clustering': 0.742, 'year': 2014, 'total_nodes': 861, 'subnetwork_sizes': [861]}
{'density': 0.11044858941670588, 'average_distance': 58.40572895898593, 'diameter': 10.0, 'average_clustering': 0.734, 'year': 2015, 'total_nodes': 1393, 'subnetwork_sizes': [1393]}
{'density': 0.09411216532167793, 'average_distance': 59.56280831618312, 'diameter': 9.0, 'average_cl